<a href="https://www.kaggle.com/code/aisuko/speech-recognition-via-whisper?scriptVersionId=222441946" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -q -U openai-whisper
!pip install -q -U setuptools-rust

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.8 MB/s eta 0:00:00


In [2]:
import whisper

model=whisper.load_model('large', device='cpu')

100%|██████████████████████████████████████| 2.88G/2.88G [00:26<00:00, 116MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [3]:
model.encoder.to('cuda:0')
model.decoder.to('cuda:1')

TextDecoder(
  (token_embedding): Embedding(51866, 1280)
  (blocks): ModuleList(
    (0-31): 32 x ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(in_features=1280, out_features=1280, bias=True)
        (key): Linear(in_features=1280, out_features=1280, bias=False)
        (value): Linear(in_features=1280, out_features=1280, bias=True)
        (out): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (cross_attn): MultiHeadAttention(
        (query): Linear(in_features=1280, out_features=1280, bias=True)
        (key): Linear(in_features=1280, out_features=1280, bias=False)
        (value): Linear(in_features=1280, out_features=1280, bias=True)
        (out): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (cross_attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=1280, out_f

In [4]:
model.decoder.register_forward_pre_hook(lambda _, inputs: tuple([inputs[0].to("cuda:1"), inputs[1].to("cuda:1")] + list(inputs[2:])))
model.decoder.register_forward_hook(lambda _, inputs, outputs: outputs.to("cuda:0"))

## The Tokenization

In [5]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 003 tokenization [7xTGNNLPyMI].webm')
print(result["text"])

 just have this giant texture of text and now we want to get neural nets that mimic it. Okay now before we plug text into neural networks we have to decide how we're going to represent this text and how we're going to feed it in. Now the way our technology works for these neural nets is that they expect a one-dimensional sequence of symbols and they want a finite set of symbols that are possible and so we have to decide what are the symbols and then we have to represent our data as one dimensional sequence of those symbols. So right now what we have is a one-dimensional sequence of text. It starts here and it goes here and then it comes here etc. So this is a one-dimensional sequence even though on my monitor of course it's laid out in a two-dimensional way but it goes from left to right and top to bottom right? So it's a one-dimensional sequence of text. Now this being computers of course there's an underlying representation here so we're going to have to figure out what the symbols a

## The Neural Network I/O

In [6]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 004 neural network I⧸O [7xTGNNLPyMI].webm')
print(result["text"])

 So, yeah, we're going to come back to tokenization, but that's for now where we are. Okay, so what I've done now is I've taken this sequence of text that we have here in the dataset, and I have re-represented it using our tokenizer into a sequence of tokens. And this is what that looks like now. So, for example, when we go back to the FindWeb dataset, they mentioned that not only is this 44 terabytes of disk space, but this is about a 15 trillion token sequence in this dataset. And so here, these are just some of the first one or two or three or a few thousand here, I think, tokens of this dataset, but there's 15 trillion here to keep in mind. And again, keep in mind one more time that all of these represent little text chunks. They're all just like atoms of these sequences, and the numbers here don't make any sense. They're just unique IDs. Okay, so now we get to the fun part, which is the neural network training. And this is... This is where a lot of the heavy lifting happens comput

## The Neural Network Internals

In [7]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 005 neural network internals [7xTGNNLPyMI].webm')
print(result["text"])

 the statistical patterns of how these tokens follow each other in the data. So let's now briefly get into the internals of these neural networks just to give you a sense of what's inside. So neural network internals. So as I mentioned we have these inputs that are sequences of tokens. In this case this is four input tokens but this can be anywhere between zero up to let's say 8,000 tokens. In principle this can be an infinite number of tokens, it would just be too computationally expensive to process an infinite number of tokens, so we just crop it at a certain length and that becomes the maximum context length of that model. Now these inputs x are mixed up in a giant mathematical expression together with the parameters or the weights of these neural networks. So here I'm showing six example parameters and their setting but in practice these modern neural networks will have billions of these parameters. And in the beginning these parameters are completely randomly set. Now with a rand

## Base Model Inference

In [8]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 008 Llama 3.1 base model inference [7xTGNNLPyMI].webm')
print(result["text"])

 and get probably a lot more coherent text than what we're seeing here a lot faster. Okay, so unfortunately, I do not have a couple of 10 or 100 million of dollars to spend on training a really big model like this. But luckily, we can turn to some big tech companies who train these models routinely and release some of them once they are done training. So they've spent a huge amount of compute to train this network, and they release the network at the end of the optimization. So it's very useful because they've done a lot of compute for that. So there are many companies who train these models routinely, but actually not many of them release these what's called base models. So the model that comes out at the end here is what's called a base model. What is a base model? It's a token simulator, right? It's an internet text token simulator. And so that is not by itself useful yet, because what we want is what's called an assistant. We want to ask questions and have it respond to answers. Th

## Pretraining and Post-Training Stage

In [9]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 009 pretraining to post-training [7xTGNNLPyMI].webm')
print(result["text"])

 Okay, so this is one way to create an assistant, even though you may only have a base model. Okay, so this is the kind of brief summary of the things we talked about over the last few minutes. Now, let me zoom out here. And this is kind of like what we've talked about so far. We wish to train LLM assistants like ChatGPT. We've discussed the first stage of that, which is the pre-training stage. And we saw that really what it comes down to is we take internet documents, we break them up into these tokens, these atoms of little text chunks, and then we predict token sequences using neural networks. The output of this entire stage is this base model. It is the setting of the parameters of this network. And this base model is basically an internet document simulator on the token level. So it can generate token sequences that have the same kind of like statistics as internet documents. And we saw that we can use it in some applications, but we actually need to do better. We want an assistan

## Post-training Data

In [10]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 010 post-training data (conversations) [7xTGNNLPyMI].webm')
print(result["text"])

 not sample internet documents but to give answers to questions. So in other words what we want to do is we want to start thinking about conversations and these are conversations that can be multi-turn so there can be multiple turns and they are in the simplest case a conversation between a human and an assistant. And so for example we can imagine the conversation could look something like this when a human says what is two plus two the assistant should respond with something like two plus two is four. When a human follows up and says what if it was star instead of a plus an assistant could respond with something like this. And similar here this is another example showing that the assistant could also have some kind of a personality here that it's kind of like nice. And then here in the third example I'm showing that when a human is asking for something that we don't wish to help with we can produce what's called a refusal. We can say that we cannot help with that. So in other words wh

## Hallucinations, Tool use and Knowledge/Working Memory

In [11]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 011 hallucinations, tool use, knowledge⧸working memory [7xTGNNLPyMI].webm')
print(result["text"])

 roughly how you can kind of think about what's happening behind the scenes here in the statistical sense. Okay, now I want to turn to the topic of LLM psychology, as I like to call it, which is what are sort of the emergent cognitive effects of the training pipeline that we have for these models. So in particular, the first one I want to talk to is, of course, hallucinations. So you might be familiar with model hallucinations. It's when LLMs make stuff up. They just totally fabricate information, etc. And it's a big problem with LLM assistants. It is a problem that existed to a large extent with early models from many years ago. And I think the problem has gotten a bit better because there are some medications that I'm going to go into in a second. For now, let's just try to understand where these hallucinations come from. So here's a specific example of three conversations that you might think you have in your training set. And these are pretty reasonable conversations that you could

## Knwoledge of Self

In [12]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 012 knowledge of self [7xTGNNLPyMI].webm')
print(result["text"])

 to summarize it and that's basically what's happening here or the equivalent of it the next sort of psychological quirk i'd like to talk about briefly is that of the knowledge of self so what i see very often on the internet is that people do something like this they ask llms something like what model are you and who built you and basically this question is a little bit nonsensical and the reason i say that is that as i try to kind of explain with some of the under the hood fundamentals this thing is not a person right it doesn't have a persistent existence in any way it sort of boots up processes tokens and shuts off and it does that for every single person it just kind of builds up a context window of conversation and then everything gets deleted and so this this entity is kind of like restarted from scratch every single conversation if that makes sense it has no persistent self there's no sense of self it's a token tumbler and it follows the statistic statistical regularities of it

## Models Need Tokens to Think

In [13]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 013 models need tokens to think [7xTGNNLPyMI].webm')
print(result["text"])

 way it's not actually like really deeply there in any real sense as it would be for a human i want to now continue to the next section which deals with the computational capabilities or like i should say the native computational capabilities of these models in problem solving scenarios and so in particular we have to be very careful with these models when we construct our examples of conversations and there's a lot of sharp edges here and that are kind of like elucidative is that a word they're kind of like interesting to look at when we consider how these models think so um consider the following prompt from a human and suppose that basically that we are building out a conversation to enter into our training set of conversations so we're going to train the model on this we're teaching you how to basically solve simple math problems so the prompt is emily buys three apples and two oranges each orange cost two dollars the total cost is 13. what is the cost of apples very simple math qu

## Tokenization Revisited

In [14]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 014 tokenization revisited： models struggle with spelling [7xTGNNLPyMI].webm')
print(result["text"])

 That's why also the models are not very good at counting. If you need them to do counting tasks, always ask them to lean on the tool. Now the models also have many other little cognitive deficits here and there and these are kind of like sharp edges of the technology to be kind of aware of over time. So as an example, the models are not very good with all kinds of spelling related tasks. They're not very good at it and I told you that we would loop back around to tokenization and the reason to do for this is that the models they don't see the characters, they see tokens and their entire world is about tokens which are these little text chunks and so they don't see characters like our eyes do and so very simple character level tasks often fail. So for example, I'm giving it a string ubiquitous and I'm asking it to print only every third character starting with the first one. So we start with u and then we should go every third so every so one two three q should be next and then etc. So

## Jagged Intelligence

In [15]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 015 jagged intelligence [7xTGNNLPyMI].webm')
print(result["text"])

 and when you're using these models in practice. I don't actually want to have a comprehensive analysis here of all the ways that the models are kind of like falling short. I just want to make the point that there are some jagged edges here and there, and we've discussed a few of them, and a few of them make sense, but some of them also will just not make as much sense. And they're kind of like you're left scratching your head, even if you understand in depth how these models work. And a good example of that recently is the following. The models are not very good at very simple questions like this. And this is shocking to a lot of people because these problems can solve complex math problems. They can answer PhD-grade physics, chemistry, biology questions much better than I can, but sometimes they fall short in super simple problems like this. So here we go. 9.11 is bigger than 9.9, and it justifies this in some way, but obviously... And then at the end, okay, it actually flips its dec

## Supervised Finetuning to Reinforcement Learning

In [16]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 016 supervised finetuning to reinforcement learning [7xTGNNLPyMI].webm')
print(result["text"])

 trust and you want to use it as a tool not as something that you kind of like let her rip on a problem and copy paste the results okay so we have now covered two major stages of training of large language models we saw that in the first stage this is called the pre training stage we are basically training on Internet documents and when you train a language model on Internet documents you get what's called a base model and it's basically an Internet document simulator right now we saw that this is interesting artifact and this takes many months to train on thousands of computers and it's kind of a lossy compression of the internet and it's extremely interesting but it's not directly useful because we don't want to sample Internet documents we want to ask questions of an AI and have it respond to our questions so for that we need an assistant and we saw that we can actually construct an assistant in the process of a post training and specifically in the process of post training we can a

## Reinforcement Learning

In [17]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 017 reinforcement learning [7xTGNNLPyMI].webm')
print(result["text"])

 stuff out and that's what reinforcement learning is about okay so let's go back to the problem that we worked with previously just so we have a concrete example to talk through as we explore sort of the topic here so um i'm here in the tech tokenizer because i'd also like to well i get a text box which is useful but number two i want to remind you again that we're always working with one-dimensional token sequences and so um i actually like prefer this view because this is like the native view of the llm if that makes sense like this is what it actually sees it sees token ids right okay so emily buys three apples and two oranges each orange is two dollars the total cost of all the fruit is thirteen dollars what is the cost of each apple and what i'd like to what i'd like you to appreciate here is these are like four possible candidate solutions as an example and they all reach the answer three now what i'd like you to appreciate at this point is that Gdolyn you already mentioned that 

## DeepSeek-R1

In [18]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 018 DeepSeek-R1 [7xTGNNLPyMI].webm')
print(result["text"])

 equivalent to the process that we train that we use for training of children the next point i would like to make is that actually these first two stages pre-training and surprise fine tuning they've been around for years and they are very standard and everyone does them all the different lm providers it is this last stage the rl training that is a lot more early in its process of development and is not standard yet in the field and so um this stage is a lot more kind of early and nascent and the reason for that is because i actually skipped over a ton of little details here in this process the high level idea is very simple it's trial and error learning but there's a ton of details and little mathematical kind of like nuances to exactly how you pick the solutions that are the best and how much you train on them and what is the prompt distribution and how to set up the training run such that this actually works so there's a lot of little details and knobs to the core idea that is very 

## AlphaGO

In [19]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 019 AlphaGo [7xTGNNLPyMI].webm')
print(result["text"])

 frontier development of pushing the performance on these very difficult problems using reasoning that is emergent in these optimizations. One more connection that I wanted to bring up is that the discovery that reinforcement learning is extremely powerful way of learning is not new to the field of AI. And one place where we've already seen this demonstrated is in the game of Go. And famously DeepMind developed the system AlphaGo, and you can watch a movie about it, where the system is learning to play the game of Go against top human players. And when we go to the paper underlying AlphaGo, so in this paper, when we scroll down we actually find a really interesting plot that I think is kind of familiar to us and we're kind of like rediscovering in the more open domain of arbitrary problem solving instead of on the closed specific domain of the game of Go. And I think that's a really interesting thing to see. And I think that's a really interesting thing to But basically what they saw, 

## Reinforcement Learning from Human Feedback

In [20]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 020 reinforcement learning from human feedback (RLHF) [7xTGNNLPyMI].webm')
print(result["text"])

 diagrams but in the domain of open thinking instead of a closed domain like Game of Go. There's one more section within reinforcement learning that I wanted to cover and that is that of learning in unverifiable domains. So far all of the problems that we've looked at are in what's called verifiable domains, that is any candidate solution we can score very easily against a concrete answer. So for example answer is three and we can very easily score these solutions against the answer of three. Either we require the models to like box in their answers and then we just check for equality of whatever's in the box with the answer or you can also use kind of what's called an LLM judge. So the LLM judge looks at a solution and it gets the answer and just basically scores the solution for whether it's consistent with the answer or not. And LLMs empirically are good enough at the current capability that they can do this fairly reliably. So we can apply those kinds of techniques as well. In any 

## Preview of Things to Come

In [21]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 021 preview of things to come [7xTGNNLPyMI].webm')
print(result["text"])

 to talk about, this is how they're trained, and this is what they are. Let's now turn to what are some of the future capabilities of these models, probably what's coming down the pipe, and also where can you find these models. I have a few bullet points on some of the things that you can expect coming down the pipe. The first thing you'll notice is that models will very rapidly become multimodal. Everything I've talked about above concerns text, but very soon we'll have LLMs that can not just handle text, but they can also operate natively and very easily over audio, so they can hear and speak, and also images, so they can see and paint. And we're already seeing the beginnings of all of this, but this will be all done natively inside the language model, and this will enable kind of like natural conversations. And roughly speaking, the reason that this is actually no different from everything we've covered above is that as a baseline, you can tokenize audio and images and apply the exa

## Keeping Track of LLMs

In [22]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 022 keeping track of LLMs [7xTGNNLPyMI].webm')
print(result["text"])

 kind of keep track of this progress and, you know, be up to date with the latest and greatest of what's happening in the field. So I would say the three resources that I have consistently used to stay up to date are number one, LLM Arena. So let me show you LLM Arena. This is basically an LLM leaderboard, and it ranks all the top models. And the ranking is based on human comparisons. So humans prompt these models, and they get to judge which one gives a better answer. They don't know which model is which. They're just looking at which model is the better answer. And you can calculate a ranking, and then you get some results. And so what you can hear is, what you can see here is the different organizations like Google Gemini, for example, that produce these models. When you click on any one of these, it takes you to the place where that model is hosted. And then here we see Google is currently on top with OpenAI right behind. Here we see DeepSeek in position number three. Now, the reas

## Where to Find LLMs

In [23]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 023 where to find LLMs [7xTGNNLPyMI].webm')
print(result["text"])

 get all your latest and greatest on X as well. So these are the major places that have worked for me over time. And finally a few words on where you can find the models and where can you use them. So the first one I would say is for any of the biggest proprietary models you just have to go to the website of that LLM provider. So for example for OpenAI that's chat.com I believe actually works now. So that's for OpenAI. Now for or you know for Gemini I think it's Gemini.google.com or AI studio. I think they have two for some reason that I don't fully understand. No one does. For the open weights models like Deepsea, Clamma, etc. you have to go to some kind of an inference provider of LLMs. So my favorite one is together.ai and I showed you that when you go to the playground of together.ai then you can sort of pick lots of different models and all of these are open models of different types and you can talk to them here as an example. Now if you'd like to use a base model like you know a

## Grand Summary

In [24]:
result=model.transcribe('/kaggle/input/download-youtube-video-with-yt-dlp/Deep Dive into LLMs like ChatGPT - 024 grand summary [7xTGNNLPyMI].webm')
print(result["text"])

 So those are a few words on where to find them. So let me now loop back around to where we started. The question was when we go to chashupt.com and we enter some kind of a query and we hit go, what exactly is happening here? What are we seeing? What are we talking to? How does this work? And I hope that this video gave you some appreciation for some of the under-the-hood details of how these models are trained and what this is that is coming back. So in particular we now know that your query is taken and is first chopped up into tokens. So we go to TickTokenizer and here where is the place in the sort of format that is for the user query, we basically put in our query right there. So our query goes into what we discussed here is the conversation protocol format which is this way that we maintain conversation objects. So this gets inserted there and then this whole thing ends up being just a token sequence. One dimensional token sequence under the hood. So chashupt saw this token seque

# Original Video

* https://www.youtube.com/watch?v=7xTGNNLPyMI

# References
* https://github.com/karpathy/llm.c/discussions/481